# Boundaries Simplification
#### The boundary files provided by the ONS can be very large.  To display within the Jupyter notbook enviroment it can help to simplify these files.This notebook will show you various techniques to simplify boundaries for display on a folium map . 

#### You will need to have enough RAM avaliable to complete these operations where the files are very large.

#### For an investigation into the boundary reduction methods and issues see https://metoffice.atlassian.net/wiki/spaces/AD/pages/3736698907/Reduce+Resolution+for+display+of+ONS+Datasets.



## Setup

In [6]:
import os

import folium
from clean_air.visualise import generate_map_based_visualisations as gmbv

#### Set up some variables defining the location of your input data.


In [7]:
SAMPLEDIR = "../cap-sample-data"  #HOW DOES THIS WORK ??

SAMPLEDIR = "/net/home/h05/clucas/Datafiles/cap-sample-data/cap-sample-data"  

boundary_data = os.path.join(SAMPLEDIR, "shapefiles","NUTS_Level_1_boundries500mgjsn.geojson")
lsoa_data = os.path.join(SAMPLEDIR, "shapefilesFullZip","Lower_Layer_Super_Output_Areas__December_2001__EW_BGC-shp.zip")
naw_regions_data = os.path.join(SAMPLEDIR, "shapefilesFullZip","Lower_Layer_Super_Output_Areas__December_2001__EW_BGC-shp.zip")

#save_path = os.path.join("assets", "tmp_output_files", "boundaries.html")
#boundary_map = gmbv.get_boundaries(boundary_data, save_path)

#### Open a zipped shapefile and inspect the contents using Geopandas

In [8]:
import geopandas
import folium
# Create base map
#boundary_map = folium.Map(location=[50.72039, -1.88092], zoom_start=7)

counties_gdf = geopandas.read_file(lsoa_data)
counties_gdf.head()


,OBJECTID,LSOA01CD,LSOA01NM,LSOA01NMW,Shape__Are,Shape__Len,geometry
0,1,E01000001,City of London 001A,City of London 001A,1.298432e+05,2370.656583,"POLYGON ((532282.630 181906.499, 532308.231 18..."
1,2,E01000002,City of London 001B,City of London 001B,2.278982e+05,2481.826139,"POLYGON ((532746.811 181786.893, 532738.483 18..."
2,3,E01000003,City of London 001C,City of London 001C,5.873326e+04,1170.012290,"POLYGON ((532293.069 182068.423, 532305.057 18..."
3,4,E01000004,City of London 001D,City of London 001D,2.292225e+06,9764.246314,"POLYGON ((533410.687 182037.862, 533403.250 18..."
4,5,E01000005,City of London 001E,City of London 001E,1.891322e+05,2202.802920,"POLYGON ((533499.066 181582.402, 533520.874 18..."



## Boundary reduction working at file level . This is sutiable for displaying smaller files.

#### Reduce boundary file using simplify function in geopandas . This uses shapley's simplify package. 1 unit represents 1 metre. No topology comsidered.

In [9]:
# Simplify function in geopandas . This uses shapley's simplify package. 1 unit represents 1 metre. No topology comsidered.simple
import geopandas
import folium
simple_map = folium.Map(location=[50.72039, -1.88092], zoom_start=7)

#Open a shapefile using geopandas
counties_gdf = geopandas.read_file(naw_regions_data)
s_gdf = counties_gdf.simplify(500)
s_gdf.dtype
s_gdf.head

folium.GeoJson(data=counties_gdf["geometry"], style_function=lambda x:{'fillColor': '#228B22', 'color': '#228B22'}, name='originalONSboundaries').add_to(simple_map)

folium.GeoJson(data=s_gdf, style_function=lambda x:{'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}, name = 'Simplify500_geopandas').add_to(simple_map)

folium.LayerControl().add_to(simple_map)
simple_map

MemoryError: 

Reduce Boundary file using topojson package, preserving topology and tolerance of 500

In [ ]:
#Reduce boundaries using the topojson package, preserving topology and tolerance of 500
boundary_map = folium.Map(location=[50.72039, -1.88092], zoom_start=7)

gdf = counties_gdf
import topojson as tp
topo = tp.Topology(gdf.to_crs({'init':'epsg:3857'}), prequantize=False)# conversion to topo using topojson.
simple = topo.toposimplify(500).to_gdf() #now simplify
simple.crs = ({'init':'epsg:3857'}) #set crs

folium.GeoJson(data=counties_gdf["geometry"], style_function=lambda x:{'fillColor': '#228B22', 'color': '#228B22'},name='originalONSboundaries').add_to(boundary_map)

folium.GeoJson(simple, style_function=lambda x:{'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}, name = 'topoJsonSimplify500').add_to(boundary_map)

folium.GeoJson(data=s_gdf, style_function=lambda x:{'fillColor': '#FF0000', 'color': '#FF0000'}, name = 'Simplify500_geopandas').add_to(boundary_map)

folium.LayerControl().add_to(boundary_map)

boundary_map

## Boundary reduction working at polygon level . This is sutiable for displaying larger files and must be run on a machine with sufficient RAM

Reduce Boundary file using Geopandas and Shapleys simplify package. This version loops through polygons to add them iteratively. Use for boundary files with many polygons to add. Will avoid RAM issues.

In [ ]:
# Loop through polygons Iteratively to add to map ..Brilliant :-) now try this method for the giant file !!! :-)..and run on spice....
import geopandas as gpd
df = geopandas.read_file('National_Assembly_for_Wales_Electoral_Regions_(December_2018)_Boundaries_WA_BSC.zip')
m  = folium.Map(location=[50.72039, -1.88092], zoom_start=7)
df = df.to_crs(epsg=4326) # set to correct projection
print(df.crs)
#df.head()

for _, r in df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['nawer18nm']).add_to(geo_j)
    geo_j.add_to(m) 
m

In [ ]:
# Loop through polygons Iteratively to add to map ..Brilliant :-) now try this method for the giant file !!! :-)..and run on spice....
import geopandas as gpd
import folium

df = gpd.read_file('Lower_Layer_Super_Output_Areas__December_2001__EW_BGC-shp.zip')
m  = folium.Map(location=[50.72039, -1.88092], zoom_start=7)
df = df.to_crs(epsg=4326) # set to correct projection
print(df.crs)
#df.head()

for _, r in df.iterrows():
    # Without simplifying the representation of each polygon,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['LSOA01NM']).add_to(geo_j)
    geo_j.add_to(m) 
    
m.save("LSOA.html") #Save my completed map
m